In [ ]:
from collections import OrderedDict
from IPython.display import display, HTML, Image
from base64 import b64encode

from rmgpy.molecule import Molecule
from rmgpy.species import Species

# Load Species Dictionary

In [ ]:
# slightly modified version of rmgpy.chemkin.loadSpeciesDictionary
def loadSpeciesDictionary(path):
    speciesDict = OrderedDict()
    with open(path, 'r') as f:
        adjlist = ''
        for line in f:
            if line.strip() == '' and adjlist.strip() != '':
                # Finish this adjacency list
                species = Species().fromAdjacencyList(adjlist)
                label = species.label
                speciesDict[label] = species
                adjlist = ''
            else:
                if "InChI" in line:
                    line = line.split()[0] + '\n'
                if '//' in line:
                    index = line.index('//')
                    line = line[0:index]
                adjlist += line
        else: #reach end of file
            if adjlist.strip() != '':
                species = Species().fromAdjacencyList(adjlist)
                label = species.label
                speciesDict[label] = species

    return speciesDict

In [ ]:
# set path here
path = '/path/to/species_dictionary.txt'

# load
species_dict = loadSpeciesDictionary(path)

# Visualize All Species

In [ ]:
html = ['<table style="width:100%;table-layout:fixed;">']
html += ['<tr><th colspan="{0}">Total {1} Species</th></tr>'.format(2, len(species_dict))]
for label, species in species_dict.iteritems():
    html += ['<tr><td colspan="{0}">{1}</td>'.format(1, label)]
    html += ['<td colspan="{0}"><img src="data:image/png;base64,{1}"></td></tr>'.format(1, b64encode(species._repr_png_()))]
html += ['</table>']

display(HTML(''.join(html)))

# Search For a Single Species

In [ ]:
# specify what you're searching for
target = Species(molecule=[Molecule().fromSMILES('CCCC')])

# target = Species(molecule=[Molecule().fromInChI('InChI=1S/C5H6/c1-2-4-5-3-1/h1-4H,5H2')])  # can use InChI instead

target.generate_resonance_structures()

In [ ]:
# search the species dictionary
for label, species in species_dict.iteritems():
    if target.isIsomorphic(species):
        print label
        display(species)
        break
else:
    print 'Species not found...'

# Visualize By Molecular Formula

In [ ]:
# sort species dictionary by formula
formula_dict = {}
for species in species_dict.itervalues():
    formula = species.molecule[0].getFormula()
    try:
        formula_dict[formula].append(species)
    except KeyError:
        formula_dict[formula] = [species]

print formula_dict.keys().sort()

In [ ]:
# view species for specified formula
desired = 'C2H4O'

html = ['<table style="width:100%;table-layout:fixed;">']
html += ['<tr><th colspan="{0}">There are {1} species with formula {2}.</th></tr>'.format(2, len(formula_dict[desired]), desired)]
for species in formula_dict[desired]:
    html += ['<tr><td colspan="{0}">{1}</td>'.format(1, species.label)]
    html += ['<td colspan="{0}"><img src="data:image/png;base64,{1}"></td></tr>'.format(1, b64encode(species._repr_png_()))]
html += ['</table>']

display(HTML(''.join(html)))